In [ ]:
import pandas as pd
import numpy as np
pred_df = pd.read_csv("/Users/christianhellum/Cand. Merc./Data-Science-Project/data_science_project/Beat the bookies/data/df_preprocessed.csv", index_col=0)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

X = pred_df.drop(['FTR', 'Date', ], 1)
Y = pred_df['FTR']

X_test = pred_df.drop(['FTR', 'Date', ], 1)
y_test = pred_df['FTR']

In [ ]:
import keras
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [ ]:
def transform_results(results):
    transformed = []
    for i in range(len(results)):
        if results[i] == 'H':
            transformed.append(0)
        elif results[i] == 'A':
            transformed.append(2)
        else:
            transformed.append(1)
    return np.array(transformed)
            
Y = transform_results(Y)
y_test = transform_results(y_test)

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
X_train_scaled = X[:-160]
y_train = Y[:-160]
X_test_scaled = X[-160:]
y_test = y_test[-160:]

In [ ]:
print('Number of matches in training data:', X_train_scaled.shape[0])
print('Number of matches in test data:', X_test_scaled.shape[0])
print('Number of features:', X_train_scaled.shape[1])

In [ ]:
# input dimension is number of features
input_dim = X_train_scaled.shape[1]

activation_func = 'relu'
kernel_init = 'glorot_normal'
batch_size = 32

model = keras.Sequential([
    Dense(48, input_shape=(input_dim,), activation=activation_func),
    Dropout(0.3),
    Dense(16),
    Dropout(0.7),
    Dense(3, activation='softmax')
])

In [ ]:
from keras.optimizers import Adam, SGD

In [ ]:
es = EarlyStopping(monitor='loss', patience=3, verbose=1)

learning_rate = 0.001


opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# Compile your model with the optimizer
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train_scaled, y_train, batch_size=batch_size, callbacks=[es], epochs=500, verbose=0)

In [ ]:
train_loss, train_acc = model.evaluate(X_train_scaled, y_train)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print('Training loss:', train_loss)
print('Training accuracy:', train_acc)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_preds = model.predict(X_test_scaled)
y_pred_argmax = [np.argmax(i) for i in y_preds]
print(classification_report(y_test, y_pred_argmax))

In [ ]:
funds = 100
wager = 10
favourites = 0
no_bets = 0
min_diff = 0.03

y_preds = model.predict(X_test_scaled)

for i in range(len(X_test_scaled)):
    prediction = np.argmax(y_preds[i])
    print('\nPrediction', prediction)
    print('Actual', y_test[i])
    print('Favourite', np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]))
    print('Prediction proba', y_preds[i])
    print('Home, Draw and Away odds', X_test['B365H'][i],       
          X_test['B365D'][i], X_test['B365A'][i])
    
    if prediction == 0:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365H'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
                
            if  prediction == y_test[i]:
                funds += (wager * X_test['B365H'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
            
    elif prediction == 1:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365D'][i])
        if odds_diff > min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
            if  prediction == y_test[i]:
                funds +=( wager * X_test['B365D'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
    else:
        odds_diff = y_preds[i][prediction] - (1/X_test['B365A'][i])
        if odds_diff >  min_diff:
            if prediction == np.argmin([X_test['B365H'][i], X_test['B365D'][i], 
                                  X_test['B365A'][i]]):
                favourites +=1
            if  prediction == y_test[i]:
                funds += (wager * X_test['B365A'][i]) - wager
            else:
                funds -= wager
        else:
            no_bets +=1
   
    print('Funds', funds)
    
print(f'Betted on favourite {favourites} times out of {len(X_test)} matches.')
print(f'No bet placed {no_bets} times')